In [165]:
import pandas as pd
import numpy as np
from sklearn import preprocessing as pp

first_match=1
second_match=20


In [166]:
# import glob
# import re
# import os
# from nltk.stem import PorterStemmer
# from nltk.stem import WordNetLemmatizer 
# from nltk.corpus import stopwords

# large_word_dict = {}
# # create dictionary based on train set
# def create_word_dict(path):
#     stemmer = PorterStemmer()
    
#     for filename in glob.glob(os.path.join(path, "*.txt")):
#         with open(filename, "r") as content_file:
#             content = content_file.read()
#             # lowercase words
#             content = np.char.lower(content)

#             # strip Punctuation
#             content = re.sub('[^\w\s]', ' ', str(content))

#             # lemmatize and remove stop Words
#             for word in content.split():
#                 if word not in stopwords.words("english"):
#                     w = stemmer.stem(word)
#                     if w in large_word_dict:
#                         large_word_dict[w] += 1
#                     else:
#                         large_word_dict[w] = 1
                        
# create_word_dict("data/descriptions_train/")

In [167]:
# # build bag of words for test and train descriptions
# word_dict = {}
# def build_bag_of_words(path, thresh):
#     index = 0
#     stemmer = PorterStemmer()

#     index = 0
#     for w in large_word_dict:
#         if large_word_dict[w] > thresh:
#             word_dict[w] = index
#             index += 1  
    
#     des_vec = {}
#     for filename in glob.glob(os.path.join(path, "*.txt")):
#         with open(filename, "r") as content_file:
#             content = content_file.read()
#             # lowercase words
#             content = np.char.lower(content)

#             # remove punctuation
#             content = re.sub('[^\w\s]', ' ', str(content))

#             # Lemmatize and remove stop words
#             cur = [0] * len(word_dict)
#             for word in content.split():
#                 try:
#                     if word not in stopwords.words("english"):
#                         w = stemmer.stem(word)
#                         if w in word_dict:
#                             cur[word_dict[w]] += 1
#                 except:
#                     pass
#             des_vec[filename.split('/')[-1]] = cur
#     return des_vec

# train_des = build_bag_of_words("data/descriptions_train/", 10)
# test_des = build_bag_of_words("data/descriptions_test/", 10)


In [168]:
# from sklearn import preprocessing as pp

# def post_process_descriptions(desc_feat):
#     for k, v in desc_feat.items():
#         post = pp.normalize(np.array(v).reshape(1, -1), norm='l2')
#         desc_feat[k] = post.flatten()

# post_process_descriptions(train_desc_features)
# post_process_descriptions(test_des)

In [173]:
# train_des_df= pd.DataFrame(train_des).T
# train_des_df['index_col'] = train_des_df.index.str.replace('.txt','').astype(int)
# train_des_df=train_des_df.sort_values('index_col')
# train_des_df=train_des_df.set_index('index_col')

# test_des_df= pd.DataFrame(test_des).T
# test_des_df['index_col'] = test_des_df.index.str.replace('.txt','').astype(int)
# test_des_df=test_des_df.sort_values('index_col')
# test_des_df=test_des_df.set_index('index_col')

# train_des_df.to_csv('train_des_df.csv',index=False)
# test_des_df.to_csv('test_des_df.csv',index=False)


train_des_df=pd.read_csv('train_des_df.csv')
test_des_df=pd.read_csv('test_des_df.csv')

In [174]:
import numpy as np; import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


res=cosine_similarity(test_des_df,train_des_df)

simmat=pd.DataFrame(list(res))
simmat=simmat.unstack().reset_index() 

simmat.columns=['train_desc','test_desc','sim']
simmat=simmat.sort_values(['test_desc', 'sim'], ascending=[True, False])
simmat['rank'] = simmat.groupby(['test_desc'])['sim'].rank(method='first',ascending=False).astype(int)

simmat_top20 = simmat[simmat['rank'] < first_match+1]
docsim=simmat_top20.drop('rank', 1)

In [175]:
tags_train=pd.read_csv('features_b_train.csv')
tags_test=pd.read_csv('features_b_test.csv')
features_train=pd.read_csv('features_a_train.csv')
features_test=pd.read_csv('features_a_test.csv')
featuresdf=pd.merge(features_train,tags_train,on='img')
featuresdf_test=pd.merge(features_test,tags_test,on='img')
featuresdf=featuresdf.sort_values('img')
featuresdf_test=featuresdf_test.sort_values('img')
Y_test2=featuresdf['img'].values
Y_train2=featuresdf_test['img'].values

X_test2=featuresdf.drop('img', 1).values
X_train2=featuresdf_test.drop('img', 1).values


import numpy as np; import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity


res=cosine_similarity(X_test2,X_train2)

simmat=pd.DataFrame(list(res))
simmat=simmat.unstack().reset_index() 

simmat.columns=['test_img','train_img','sim']
simmat=simmat.sort_values(['train_img', 'sim'], ascending=[True, False])
simmat['rank'] = simmat.groupby(['train_img'])['sim'].rank(method='first',ascending=False).astype(int)

simmat_top20 = simmat[simmat['rank'] < second_match+1]
imgsim=simmat_top20.drop('rank', 1)

In [176]:
final_simmat=pd.merge(docsim,imgsim,left_on='train_desc',right_on='train_img')

final_simmat=final_simmat.sort_values(['test_desc', 'sim_x','sim_y'], ascending=[True, False,False])

final_simmat['rank'] = final_simmat.groupby(['test_desc'])['sim_x'].rank(method='first',ascending=False).astype(int)

final_simmat=final_simmat.drop('sim_x', 1)
final_simmat=final_simmat.drop('sim_y', 1)
final_simmat=final_simmat.drop('train_desc', 1)
final_simmat=final_simmat.drop('train_img', 1)

final_simmat['Descritpion_ID']=final_simmat['test_desc']
final_simmat['test_img']=final_simmat['test_img'].map(str)+'.jpg'

final_simmat=final_simmat.drop('test_desc', 1)

final_simmat=final_simmat.pivot(index='Descritpion_ID', columns='rank', values='test_img')
final_simmat.columns = ['f_'+ str(col)  for col in final_simmat.columns]

final_simmat["Descritpion_ID"] = final_simmat.index.map(str)+'.txt'
final_simmat["Top_20_Image_IDs"] = final_simmat["f_1"].map(str) + ' ' + final_simmat["f_2"].map(str) + ' '+final_simmat["f_3"].map(str) + ' '+final_simmat["f_4"].map(str) + ' '+final_simmat["f_5"].map(str) + ' '+final_simmat["f_6"].map(str) + ' '+final_simmat["f_7"].map(str) + ' '+final_simmat["f_8"].map(str) + ' ' +final_simmat["f_9"].map(str) + ' '+final_simmat["f_10"].map(str) + ' '+final_simmat["f_11"].map(str) + ' '+final_simmat["f_12"].map(str) + ' '+final_simmat["f_13"].map(str) + ' '+final_simmat["f_14"].map(str) + ' '+final_simmat["f_15"].map(str) + ' '+final_simmat["f_16"].map(str) + ' '+final_simmat["f_17"].map(str) + ' '+final_simmat["f_18"].map(str) + ' '+final_simmat["f_19"].map(str) + ' '+final_simmat["f_20"].map(str) 

submission = final_simmat[['Descritpion_ID','Top_20_Image_IDs']]
submission.to_csv('submission.csv',index=False)



In [177]:
submission

,Descritpion_ID,Top_20_Image_IDs
Descritpion_ID,,
0,0.txt,222.jpg 105.jpg 1980.jpg 1396.jpg 829.jpg 106....
1,1.txt,1714.jpg 589.jpg 14.jpg 753.jpg 885.jpg 1806.j...
2,2.txt,838.jpg 1724.jpg 634.jpg 1183.jpg 159.jpg 696....
3,3.txt,1035.jpg 1609.jpg 1315.jpg 1513.jpg 75.jpg 176...
4,4.txt,375.jpg 1003.jpg 995.jpg 1388.jpg 1561.jpg 292...
5,5.txt,1069.jpg 1384.jpg 1254.jpg 50.jpg 1480.jpg 116...
6,6.txt,429.jpg 414.jpg 718.jpg 623.jpg 262.jpg 1660.j...
7,7.txt,1270.jpg 1900.jpg 685.jpg 1328.jpg 1857.jpg 13...
8,8.txt,537.jpg 1942.jpg 727.jpg 1399.jpg 1770.jpg 162...


In [178]:
final_simmat

,f_1,f_2,f_3,f_4,f_5,f_6,f_7,f_8,f_9,f_10,...,f_13,f_14,f_15,f_16,f_17,f_18,f_19,f_20,Descritpion_ID,Top_20_Image_IDs
Descritpion_ID,,,,,,,,,,,,,,,,,,,,,
0,222.jpg,105.jpg,1980.jpg,1396.jpg,829.jpg,106.jpg,1883.jpg,309.jpg,82.jpg,279.jpg,...,1627.jpg,305.jpg,597.jpg,1581.jpg,1144.jpg,46.jpg,669.jpg,1886.jpg,0.txt,222.jpg 105.jpg 1980.jpg 1396.jpg 829.jpg 106....
1,1714.jpg,589.jpg,14.jpg,753.jpg,885.jpg,1806.jpg,457.jpg,1809.jpg,1054.jpg,1929.jpg,...,331.jpg,1199.jpg,1733.jpg,1550.jpg,355.jpg,1753.jpg,1507.jpg,1812.jpg,1.txt,1714.jpg 589.jpg 14.jpg 753.jpg 885.jpg 1806.j...
2,838.jpg,1724.jpg,634.jpg,1183.jpg,159.jpg,696.jpg,579.jpg,1840.jpg,445.jpg,600.jpg,...,904.jpg,953.jpg,42.jpg,720.jpg,1866.jpg,496.jpg,1107.jpg,511.jpg,2.txt,838.jpg 1724.jpg 634.jpg 1183.jpg 159.jpg 696....
3,1035.jpg,1609.jpg,1315.jpg,1513.jpg,75.jpg,1769.jpg,1850.jpg,243.jpg,66.jpg,1755.jpg,...,397.jpg,799.jpg,1404.jpg,1185.jpg,1818.jpg,654.jpg,1394.jpg,432.jpg,3.txt,1035.jpg 1609.jpg 1315.jpg 1513.jpg 75.jpg 176...
4,375.jpg,1003.jpg,995.jpg,1388.jpg,1561.jpg,292.jpg,434.jpg,736.jpg,1131.jpg,21.jpg,...,484.jpg,1856.jpg,336.jpg,1451.jpg,1135.jpg,862.jpg,1219.jpg,939.jpg,4.txt,375.jpg 1003.jpg 995.jpg 1388.jpg 1561.jpg 292...
5,1069.jpg,1384.jpg,1254.jpg,50.jpg,1480.jpg,1161.jpg,814.jpg,1510.jpg,643.jpg,526.jpg,...,413.jpg,846.jpg,79.jpg,1968.jpg,379.jpg,1592.jpg,1982.jpg,140.jpg,5.txt,1069.jpg 1384.jpg 1254.jpg 50.jpg 1480.jpg 116...
6,429.jpg,414.jpg,718.jpg,623.jpg,262.jpg,1660.jpg,655.jpg,1874.jpg,1151.jpg,489.jpg,...,1655.jpg,1265.jpg,458.jpg,1289.jpg,897.jpg,1311.jpg,1275.jpg,1815.jpg,6.txt,429.jpg 414.jpg 718.jpg 623.jpg 262.jpg 1660.j...
7,1270.jpg,1900.jpg,685.jpg,1328.jpg,1857.jpg,136.jpg,975.jpg,1239.jpg,213.jpg,1961.jpg,...,528.jpg,1285.jpg,1488.jpg,627.jpg,481.jpg,689.jpg,987.jpg,1844.jpg,7.txt,1270.jpg 1900.jpg 685.jpg 1328.jpg 1857.jpg 13...
8,537.jpg,1942.jpg,727.jpg,1399.jpg,1770.jpg,162.jpg,781.jpg,991.jpg,1048.jpg,823.jpg,...,1259.jpg,692.jpg,1386.jpg,328.jpg,1808.jpg,1548.jpg,610.jpg,401.jpg,8.txt,537.jpg 1942.jpg 727.jpg 1399.jpg 1770.jpg 162...
